In [12]:
import pandas_gbq as gbq
import re
# import texthero as hero
from IPython.core.debugger import set_trace

In [2]:
proj_id = 'sideb-proejct'
table_id = 'moiber_20210920.twitter_feed'

In [3]:
sql = '''
SELECT id, content
FROM `sideb-proejct.moiber_20210920.twitter_feed`
'''

feeds = gbq.read_gbq(sql, project_id=proj_id)

Downloading: 100%|██████████| 215003/215003 [00:53<00:00, 4029.91rows/s]


In [6]:
feeds.head()

,id,content
0,1371320157453250563,@DOYOUNGbr_ @treasuremembers Doyoung fashion\n...
1,1370918210216763392,"화장품, 마스크, 세안용클렌징, 손소독제, 라면, 곡물스낵, 그외 식품, 의류 등등..."
2,1370532027770892290,"방탄소년단 인도네시아 팬들, 의류 3500벌 수집 “제이홉 이름으로 저소득층에 기부..."
3,1371431339652640772,님들님들 의류브랜드인데 이거 이름이 뭐지 그 대각선으로 오프화이트 그래 이거야
4,1387353868288204800,ADOY의 LOVE I VIVID I CATNIP \n3개의 앨범을 의류&amp;가...


In [21]:
feeds.sample(n=100, random_state=1).to_excel('feed samples for annotation.xlsx')

In [11]:
def preproc(text, remove_url=True, remove_mention=True, remove_hashtag=False):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = '((?: rt)|(?:^rt))[^ @]?'
    EMOJI = r'[\U00010000-\U0010ffff]'
    DOTS = '…'
    LONG_BLANK = r'[ ]+'
    SPECIALS = r'([^ a-zA-Z0-9_\u3131-\u3163\uac00-\ud7a3]+)|([ㄱ-ㅣ]+)'
    
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
    URL = '|'.join((URL1, URL2))
    
    HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
    MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
    text = text.lower()
    
    if remove_url:
        text = re.sub(URL, ' ', text)

    if remove_mention:
        text = re.sub(MENTION, ' ', text)        
    else:
        text = re.sub(MENTION, ' \g<inner_mention>', text)
        
    if remove_hashtag:
        text = re.sub(HASHTAG, ' ', text)
    else:
        text = re.sub(HASHTAG, ' \g<inner_hashtag>', text)
        
    text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS)), ' ', text)
    return re.sub(LONG_BLANK, ' ', text).strip()

In [14]:
feeds['text'] = feeds['content'].map(lambda t: preproc(t))

In [15]:
feeds.head()

,id,content,text
0,1371320157453250563,@DOYOUNGbr_ @treasuremembers Doyoung fashion\n...,doyoung fashion 트레저 도영 보컬 트레저 도영 작곡가 트레저 도영 메인...
1,1370918210216763392,"화장품, 마스크, 세안용클렌징, 손소독제, 라면, 곡물스낵, 그외 식품, 의류 등등...",화장품 마스크 세안용클렌징 손소독제 라면 곡물스낵 그외 식품 의류 등등 수입해가지 ...
2,1370532027770892290,"방탄소년단 인도네시아 팬들, 의류 3500벌 수집 “제이홉 이름으로 저소득층에 기부...",방탄소년단 인도네시아 팬들 의류 3500벌 수집 제이홉 이름으로 저소득층에 기부 출...
3,1371431339652640772,님들님들 의류브랜드인데 이거 이름이 뭐지 그 대각선으로 오프화이트 그래 이거야,님들님들 의류브랜드인데 이거 이름이 뭐지 그 대각선으로 오프화이트 그래 이거야
4,1387353868288204800,ADOY의 LOVE I VIVID I CATNIP \n3개의 앨범을 의류&amp;가...,adoy의 love i vivid i catnip 3개의 앨범을 의류 amp 가방 ...


,id,content,text
8013,1388010598634381314,@TeamLTY Taeyong Fashion 태용 패션 \n#태용 #NCT태용 #T...,taeyong fashion 태용 패션 태용 nct태용 taeyong
210800,1381142543727874048,샀어\n비비안웨스트우드 늘 갖고싶었는데\n기분짱!!!!!! https://t.co/...,샀어 비비안웨스트우드 늘 갖고싶었는데 기분짱
118682,1414061813315084289,제비표비옷 추천 가성비 남성의류 가격비교 제품 TOP 50종 https://t.co...,제비표비옷 추천 가성비 남성의류 가격비교 제품 top 50종
